In [1]:
#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////// Libraries  ////////////////////////////////////////////////////////
import dpkt
from dpkt.compat import compat_ord
import os 
import pandas as pd
import socket
import timeit
from timeit import default_timer as timer
from datetime import timedelta
import datetime
import openpyxl
import numpy as np
from matplotlib import pyplot as plt
import re
import numpy as np
from datetime import datetime

In [2]:
folder = 'sample_data/normal_behavior/week_0/'
network_file = 'devices_tshark.csv'
network_results_path = folder+network_file

In [3]:
packets_pd = pd.read_csv(network_results_path, names=['Timestamp', 'MAC src', 'MAC dst', 'Length', 'IP src', 'IP dst', 'Protocol', 
                                                    'Flags', 'Info']) 
packets_pd = packets_pd[packets_pd['Protocol'] != 'ICMPv6']

In [4]:
packets_pd

,Timestamp,MAC src,MAC dst,Length,IP src,IP dst,Protocol,Flags,Info
2,1.614657e+09,00:00:00:00:00:04,ff:ff:ff:ff:ff:ff,42,NaN,NaN,ARP,NaN,Who has 192.168.1.254? Tell 192.168.1.1
3,1.614657e+09,2a:13:13:c5:41:2a,00:00:00:00:00:04,42,NaN,NaN,ARP,NaN,192.168.1.254 is at 2a:13:13:c5:41:2a
4,1.614657e+09,00:00:00:00:00:04,2a:13:13:c5:41:2a,74,192.168.1.1,10.0.2.1,TCP,0x00000002,47848 → 44818 [SYN] Seq=0 Win=42340 Len=0 MSS=...
5,1.614657e+09,8a:52:06:1b:d6:19,ff:ff:ff:ff:ff:ff,42,NaN,NaN,ARP,NaN,Who has 192.168.1.1? Tell 192.168.1.254 (dupli...
6,1.614657e+09,00:00:00:00:00:05,8a:52:06:1b:d6:19,42,NaN,NaN,ARP,NaN,192.168.1.1 is at 00:00:00:00:00:05 (duplicate...
...,...,...,...,...,...,...,...,...,...
189139,1.614658e+09,62:93:82:8b:e3:fe,00:00:00:00:00:0c,130,10.0.5.1,192.168.1.1,CIP CM,0x00000018,Connection Manager - Unconnected Send: 'T7:1' ...
189140,1.614658e+09,00:00:00:00:00:0c,62:93:82:8b:e3:fe,116,192.168.1.1,10.0.5.1,CIP,0x00000018,Success: 'T7:1' - Service (0x4c)
189141,1.614658e+09,62:93:82:8b:e3:fe,00:00:00:00:00:0c,66,10.0.5.1,192.168.1.1,TCP,0x00000011,37532 → 44818 [FIN
189142,1.614658e+09,00:00:00:00:00:0c,62:93:82:8b:e3:fe,66,192.168.1.1,10.0.5.1,TCP,0x00000011,44818 → 37532 [FIN


In [5]:
packets_pd.iloc[7]

Timestamp           1.61466e+09
MAC src       2a:13:13:c5:41:2a
MAC dst       00:00:00:00:00:04
Length                       74
IP src                 10.0.2.1
IP dst              192.168.1.1
Protocol                    TCP
Flags                0x00000012
Info         44818 → 47848 [SYN
Name: 9, dtype: object

In [6]:
def get_flag_time(a_mac, a_flag):    
# Mask with the expression we need    
    exp = (packets_pd['Flags'] == a_flag) & (packets_pd['MAC src'] == a_mac)    
    packets_pd.loc[exp,'Flag Time'] = packets_pd.loc[exp, 'Timestamp'] - packets_pd.loc[exp, 'Timestamp'].shift(1)

In [7]:
unique_pkts = packets_pd[~packets_pd.duplicated(subset=['MAC src','Flags'],keep='first')]
for i in range(len(unique_pkts.index)):
    get_flag_time(unique_pkts.iloc[i]['MAC src'], unique_pkts.iloc[i]['Flags'])
packets_pd['Flag Time'] = packets_pd['Flag Time'].fillna(0)
packets_pd['IP src'] = packets_pd['IP src'].fillna('0.0.0.0')
packets_pd['IP dst'] = packets_pd['IP dst'].fillna('255.255.255.0')
packets_pd['Flags'] = packets_pd['Flags'].fillna('0')

In [8]:
# We process the ARP packets to obtain their command
exp = packets_pd['Protocol'] == 'ARP'
packets_pd.loc[exp,'Command'] = packets_pd.loc[exp, 'Info'].apply(lambda x: 'ARP Request' if "Who has" in x else 'ARP Response')

exp = packets_pd['Protocol'] == 'ENIP'
packets_pd.loc[exp,'Command'] = packets_pd.loc[exp, 'Info']

# For now we do it this way. In the final version, we will hot encode based on flags (For Tcp packets Flags == Command)
exp = packets_pd['Protocol'] == 'TCP'
packets_pd.loc[exp,'Command'] = packets_pd.loc[exp, 'Info'].str.extract(r'\[(.*?)\]')[0]

exp = packets_pd['Protocol'] == 'CIP CM'
packets_pd.loc[exp,'Command'] = packets_pd.loc[exp, 'Info'].str.extract(r'\-(.*?)\-')[0]

exp = packets_pd['Protocol'] == 'CIP'
packets_pd.loc[exp,'Command'] = packets_pd.loc[exp, 'Info'].str.extract(r'(.*?)\-')[0]

In [9]:
packets_pd

,Timestamp,MAC src,MAC dst,Length,IP src,IP dst,Protocol,Flags,Info,Flag Time,Command
2,1.614657e+09,00:00:00:00:00:04,ff:ff:ff:ff:ff:ff,42,0.0.0.0,255.255.255.0,ARP,0,Who has 192.168.1.254? Tell 192.168.1.1,0.000000,ARP Request
3,1.614657e+09,2a:13:13:c5:41:2a,00:00:00:00:00:04,42,0.0.0.0,255.255.255.0,ARP,0,192.168.1.254 is at 2a:13:13:c5:41:2a,0.000000,ARP Response
4,1.614657e+09,00:00:00:00:00:04,2a:13:13:c5:41:2a,74,192.168.1.1,10.0.2.1,TCP,0x00000002,47848 → 44818 [SYN] Seq=0 Win=42340 Len=0 MSS=...,0.000000,SYN
5,1.614657e+09,8a:52:06:1b:d6:19,ff:ff:ff:ff:ff:ff,42,0.0.0.0,255.255.255.0,ARP,0,Who has 192.168.1.1? Tell 192.168.1.254 (dupli...,0.000000,ARP Request
6,1.614657e+09,00:00:00:00:00:05,8a:52:06:1b:d6:19,42,0.0.0.0,255.255.255.0,ARP,0,192.168.1.1 is at 00:00:00:00:00:05 (duplicate...,0.000000,ARP Response
...,...,...,...,...,...,...,...,...,...,...,...
189139,1.614658e+09,62:93:82:8b:e3:fe,00:00:00:00:00:0c,130,10.0.5.1,192.168.1.1,CIP CM,0x00000018,Connection Manager - Unconnected Send: 'T7:1' ...,0.017167,Unconnected Send: 'T7:1'
189140,1.614658e+09,00:00:00:00:00:0c,62:93:82:8b:e3:fe,116,192.168.1.1,10.0.5.1,CIP,0x00000018,Success: 'T7:1' - Service (0x4c),0.029485,Success: 'T7:1'
189141,1.614658e+09,62:93:82:8b:e3:fe,00:00:00:00:00:0c,66,10.0.5.1,192.168.1.1,TCP,0x00000011,37532 → 44818 [FIN,0.955110,NaN
189142,1.614658e+09,00:00:00:00:00:0c,62:93:82:8b:e3:fe,66,192.168.1.1,10.0.5.1,TCP,0x00000011,44818 → 37532 [FIN,0.958502,NaN


In [10]:
packets_pd.to_csv(folder+ 'parsed.csv')